In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import random

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

entangled_qbits = []
A_op_choices = []
B_op_choices = []
final_key = []

root2 = math.sqrt(2)
root3 = math.sqrt(3)

unitary_rand = [[1/root3, root2/root3],
           [root2/root3, -1/root3]]
op_V = [[(1/root2)*-1, (1/root2)*1],
        [(1/root2)*1, (1/root2)*1]]
op_W = [[(1/root2)*-1, (1/root2)*-1],
        [(1/root2)*-1, (1/root2)*1]]

rand_op = Operator(unitary_rand)
V_trans = Operator(op_V)
W_trans = Operator(op_W)

def random():
    q = QuantumCircuit(1) 
    q.unitary(rand_op, [0])
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    return counts.get("1",0)

def random_state():
    number_of_runs = 0
    state = -1
    while True:
        result = random()
        if result == 0:
            return number_of_runs%3 + 1
        else:
            number_of_runs += 1

def entangledPair():
    q = QuantumCircuit(2) 
    q.h(0)
    q.cx(0,1)
    return q

final_key = []
N=900

result_counts = {"11" : {"total": 0,
                         "instances": 0},
                 "13" : {"total": 0,
                         "instances": 0},
                 "31" : {"total": 0,
                         "instances": 0},
                 "33" : {"total": 0,
                         "instances": 0},
                }

for i in range(N):

    #create q circuit with 2 qubits
    circuit = entangledPair()

    #alice choice
    A_basis_choice = random_state()
    A_op_choices.append(A_basis_choice)
    match A_basis_choice:
        case 1: #measuring in X basis
            circuit.h(0)
        case 2: #measuring W basis
            circuit.unitary(W_trans, [0])
        case 3: #measuring Z basis
            pass

    #bob
    B_basis_choice = random_state()
    B_op_choices.append(B_basis_choice)
    match B_basis_choice:
        case 1: #measuring W basis
            circuit.unitary(W_trans, [1])
        case 2: #measuring Z basis
            pass
        case 3: #measuring V basis
            circuit.unitary(V_trans, [1])
        
    circuit.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    for key, value in counts.items():
        results = list(key)
        
    converted_bits = []
    for bit in results:
        converted_bits.append(1 if bit == "0" else -1)
        
    if (A_basis_choice==2 and B_basis_choice==1) or (A_basis_choice==3 and B_basis_choice==2):
        final_key.append(results[0]) #alice bit
    else:
        to_add = converted_bits[0] * converted_bits[1]
        if (A_basis_choice==1 and B_basis_choice==1):
            result_counts["11"]["total"] += to_add
            result_counts["11"]["instances"] += 1
        elif (A_basis_choice==1 and B_basis_choice==3):
            result_counts["13"]["total"] += to_add
            result_counts["13"]["instances"] += 1
        elif (A_basis_choice==3 and B_basis_choice==1):
            result_counts["31"]["total"] += to_add
            result_counts["31"]["instances"] += 1
        elif (A_basis_choice==3 and B_basis_choice==3):
            result_counts["33"]["total"] += to_add
            result_counts["33"]["instances"] += 1

averages = {"11": 0,
            "13": 0,
            "31": 0,
            "33": 0}

for key, value in averages.items():
    averages[key] = result_counts[key]["total"]/result_counts[key]["instances"]

S = abs(averages["11"]-averages["13"]+averages["31"]+averages["33"])
print(f"{S=}")

print(f"Alice basis choices: {A_op_choices}")
print(f"Bob basis choices: {B_op_choices}")
print(f"Shared key: {final_key}")
print(f"Final key length: {len(final_key)} out of {N} iterations")